In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from train_and_evaluate import predict_from_checkpoint_with_labels
from training_utils import train_models_by_params

import pandas as pd
from sklearn.metrics import roc_auc_score
from transforms import *


In [3]:
train_set = pd.read_csv('../data/train_set.csv').values.flatten().tolist()
val_set = pd.read_csv('../data/val_set.csv').values.flatten().tolist()

ADC_dir = ['../ext_storage/mri_data/preprocessed/DTI_eddy_MD_wm']

In [5]:
runs = [
    { # Version 0
    'img_dirs': ADC_dir,
    'transforms': TRAIN_TRANSFORMS_MIRROR_PROB,
    'basic_block_depth': 4,

    'total_epochs': 10,
    'batch_size': 4,
    'learning_rate': 3e-4,
},
]

train_models_by_params(
    train_set, 
    runs,
    validation_set = val_set
)

{'basic_block_depth': 4,
 'batch_size': 4,
 'img_dirs': ['../ext_storage/mri_data/preprocessed/DTI_eddy_MD_wm'],
 'learning_rate': 0.0003,
 'total_epochs': 10,
 'transforms': [Mirror()]}
standard RUN version_2

Loaded 291 images each with shape (1, 96, 112, 96)


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type           | Params
-----------------------------------------------
0 | net         | Basic3DCNN     | 425 K 
1 | train_acc   | BinaryAccuracy | 0     
2 | train_auroc | BinaryAUROC    | 0     
3 | val_acc     | BinaryAccuracy | 0     
4 | val_auroc   | BinaryAUROC    | 0     
5 | test_acc    | BinaryAccuracy | 0     
6 | test_auroc  | BinaryAUROC    | 0     
-----------------------------------------------
425 K     Trainable params
0         Non-trainable p

Loaded 41 images each with shape (1, 96, 112, 96)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


Model trained from 2023-10-14 09:32 to 2023-10-14 09:34
Time elapsed (hh:mm:ss.ms) 0:01:50.089171

####################################################################################



In [4]:
# Testing basic

preds, test_labels = predict_from_checkpoint_with_labels(
    checkpoint_path='../ext_storage/saved_models_storage/lightning_logs/version_0/checkpoints/last.ckpt',
    subject_list=val_set,
    img_dirs=ADC_dir,
    net_architecture='basic',
    basic_block_depth=4,
    label_csv_columns = {'subject_col': 'studyid', 'label_col': 'primary_all'},
    additional_feature_cols = None,
)

print(roc_auc_score(test_labels, preds))

Loaded 41 images each with shape (1, 96, 112, 96)


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

Model run from 2023-10-14 09:55 to 2023-10-14 09:55
Time elapsed (hh:mm:ss.ms) 0:00:03.680176
0.732843137254902


In [6]:
from monai.networks.nets.densenet import DenseNet121

In [7]:
# Testing dense

preds, test_labels = predict_from_checkpoint_with_labels(
    checkpoint_path='../ext_storage/saved_models_storage/lightning_logs_pre_9_18_23/version_281/checkpoints/last.ckpt',
    subject_list=val_set,
    img_dirs=['../ext_storage/mri_data/preprocessed/DTI_eddy_MD_wm'],
    net_architecture='densenet',
    densenet_class=DenseNet121,
    label_csv_columns = {'subject_col': 'studyid', 'label_col': 'primary_all'},
    additional_feature_cols = None,
)

print(roc_auc_score(test_labels, preds))

Loaded 41 images each with shape (1, 96, 112, 96)


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

Model run from 2023-10-14 09:56 to 2023-10-14 09:56
Time elapsed (hh:mm:ss.ms) 0:00:07.879368
0.7254901960784315


In [10]:
base_path = '../ext_storage/saved_models_storage/lightning_logs_pre_9_18_23/version_{}/checkpoints/last.ckpt'

basic_ADC_pretrained_path = base_path.format(6)
basic_T1_pretrained_path = base_path.format(21)
basic_T2_pretrained_path = base_path.format(36)
basic_trace_pretrained_path = base_path.format(51)

densenet_ADC_pretrained_path = base_path.format(281)
densenet_T1_pretrained_path = base_path.format(291)
densenet_T2_pretrained_path = base_path.format(301)
densenet_trace_pretrained_path = base_path.format(311)

LR_pretrained_path = base_path.format(366)


ensemble_model_params = {
    'cnn_model_list': [
        ## Basic
        # ADC
        {
            'pretrained_path': basic_ADC_pretrained_path, # Pretrained paths are optional
            'net_architecture': 'basic',
            'basic_block_depth': 4,
            'dropout': 0.3,
        },
        # T1
        {
            'pretrained_path': basic_T1_pretrained_path,
            'net_architecture': 'basic',
            'basic_block_depth': 4,
            'dropout': 0.3,
        },
        # T2
        {
            'pretrained_path': basic_T2_pretrained_path,
            'net_architecture': 'basic',
            'basic_block_depth': 4,
            'dropout': 0.3,
        },
        # T3
        {
            'pretrained_path': basic_trace_pretrained_path,
            'net_architecture': 'basic',
            'basic_block_depth': 4,
            'dropout': 0.3,
        },

        ## Dense
        {
            'pretrained_path': densenet_ADC_pretrained_path, # Pretrained paths are optional
            'net_architecture': 'densenet',
            'densenet_class': DenseNet121,
        },
        # T1
        {
            'pretrained_path': densenet_T1_pretrained_path,
            'net_architecture': 'densenet',
            'densenet_class': DenseNet121,
        },
        # T2
        {
            'pretrained_path': densenet_T2_pretrained_path,
            'net_architecture': 'densenet',
            'densenet_class': DenseNet121,
        },
        # T3
        {
            'pretrained_path': densenet_trace_pretrained_path,
            'net_architecture': 'densenet',
            'densenet_class': DenseNet121,
        },
    ],
    'cnn_image_index_list': [
        0, 1, 2, 3, 0, 1, 2, 3
    ], # For each corresponding model in above list

    'lr_model_list': [
        {
            'pretrained_path': LR_pretrained_path,
            'net_architecture': 'logistic_regression',
        },
    ],
}

In [11]:
# Testing dense

preds, test_labels = predict_from_checkpoint_with_labels(
    checkpoint_path='../ext_storage/saved_models_storage/lightning_logs_pre_9_18_23/version_381/checkpoints/last.ckpt',
    subject_list=val_set,
    img_dirs=['../ext_storage/mri_data/preprocessed/DTI_eddy_MD_wm',
              '../ext_storage/mri_data/preprocessed/T1_wm',
              '../ext_storage/mri_data/preprocessed/T2_wm',
              '../ext_storage/mri_data/preprocessed/DTI_eddy_trace_wm'],
    net_architecture='ensemble',
    ensemble_model_params=ensemble_model_params,
    label_csv_columns = {'subject_col': 'studyid', 'label_col': 'primary_all'},
    additional_feature_cols = ['sex', 'txtassign', 'inf_gestage_zscore', 'total_brain_injury_volume_zscore', 'inf_gestage_minmax', 'total_brain_injury_volume_minmax'],
)

print(roc_auc_score(test_labels, preds))

Loaded 41 images each with shape (4, 96, 112, 96)


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

Model run from 2023-10-14 09:56 to 2023-10-14 09:57
Time elapsed (hh:mm:ss.ms) 0:00:19.684135
0.9877450980392157
